# ETL Operations

In [1]:
import scipy, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

In [2]:
#Read the raw datasets

basedir = 'C:/Users/Traian Fulea/Desktop/Final Assessment/Data/'
df15 = pd.read_excel(os.path.join(basedir, '2015.xls'), sheet_name = 'FEguide')
df16 = pd.read_excel(os.path.join(basedir, '2016.xlsx'), sheet_name = 'FEguide')
df17 = pd.read_excel(os.path.join(basedir, '2017.xlsx'), sheet_name = 'FEguide')
df18 = pd.read_excel(os.path.join(basedir, '2018.xlsx'), sheet_name = 'FEguide')
df19 = pd.read_excel(os.path.join(basedir, '2019.xlsx'), sheet_name = '2019 FEguide')
df20 = pd.read_excel(os.path.join(basedir, '2020.xlsx'), sheet_name = '20')
df21 = pd.read_excel(os.path.join(basedir, '2021.xlsx'), sheet_name = '21')
df22 = pd.read_excel(os.path.join(basedir, '2022.xlsx'), sheet_name = '22')

frames = [df15, df16, df17, df18, df19, df20, df21, df22]

In [3]:
#Display some info about the 2015 datasets

df15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Model Year                729 non-null    int64         
 1   Mfr Name                  729 non-null    object        
 2   Division                  729 non-null    object        
 3   Carline                   729 non-null    object        
 4   Engine Displacement       729 non-null    float64       
 5   # Cylinders               729 non-null    int64         
 6   Transmission              729 non-null    object        
 7   City FE                   729 non-null    int64         
 8   Highway FE                729 non-null    int64         
 9   Combined FE               729 non-null    int64         
 10  Air Aspiration Method     729 non-null    object        
 11  Transmission Description  729 non-null    object        
 12  # Gears               

In [4]:
df16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Columns: 162 entries, Model Year to Comb PHEV Composite MPGe
dtypes: datetime64[ns](1), float64(72), int64(16), object(73)
memory usage: 1.5+ MB


## 1. Column matching and renaming

In [5]:
columns = df15.columns.tolist()
print(columns)
print(len(columns))

['Model Year', 'Mfr Name', 'Division', 'Carline', 'Engine Displacement', '# Cylinders', 'Transmission', 'City FE', 'Highway FE', 'Combined FE', 'Air Aspiration Method', 'Transmission Description', '# Gears', 'Drive Desc', 'Carline Class Desc', 'Release Date', 'City CO2', 'Highway CO2', 'Combined CO2']
19


In [6]:
#Visualize the common columns

for i in df15.columns.tolist():
    if i in df16.columns.tolist():
        print(i)

Model Year
Mfr Name
Division
Carline
Transmission
# Gears
Drive Desc
Carline Class Desc
Release Date


In [7]:
#Visualize the column names between the previous datasets

distinct = list()
same = list()
remained = columns.copy()
for i in df16.columns.tolist():
    if i not in columns:
        distinct.append(i)
    else:
        same.append(i)
        remained.remove(i)

print("Distinct column names: \n", distinct)

print("Columns that need to be found names: \n", remained)

Distinct column names: 
 ['Verify Mfr Cd', 'Index (Model Type Index)', 'Eng Displ', '# Cyl', 'City FE (Guide) - Conventional Fuel', 'Hwy FE (Guide) - Conventional Fuel', 'Comb FE (Guide) - Conventional Fuel', 'City Unadj FE - Conventional Fuel', 'Hwy Unadj FE - Conventional Fuel', 'Comb Unadj FE - Conventional Fuel', 'City Unrd Adj FE - Conventional Fuel', 'Hwy Unrd Adj FE - Conventional Fuel', 'Comb Unrd Adj FE - Conventional Fuel', 'Guzzler? ', 'Air Aspir Method', 'Air Aspiration Method Desc', 'Trans', 'Trans Desc', 'Trans, Other', 'Lockup Torque Converter', 'Trans Creeper Gear', 'Drive Sys', 'Max Ethanol % - Gasoline', 'Max Biodiesel %', 'Range1 - Model Type Driving Range - Conventional Fuel', 'Fuel Usage  - Conventional Fuel', 'Fuel Usage Desc - Conventional Fuel', 'Fuel Unit - Conventional Fuel', 'Fuel Unit Desc - Conventional Fuel', 'Gas Guzzler Exempt (Where Truck = 1975 NHTSA truck definition)', 'Gas Guzzler Exempt Desc (Where Truck = 1975 NHTSA truck definition)', '2Dr Pass Vo

In [8]:
#Rename the columns so they can match with previous datasets

for i in frames:
    if i is not df15:
        i.rename(columns = {'Eng Displ' : 'Engine Displacement',
                            '# Cyl' : '# Cylinders',
                            'City FE (Guide) - Conventional Fuel' : 'City FE',
                            'Hwy FE (Guide) - Conventional Fuel' : 'Highway FE',
                            'Comb FE (Guide) - Conventional Fuel' : 'Combined FE',
                            'Air Aspir Method' : 'Air Aspiration Method',
                            'Trans Desc' : 'Transmission Description',
                            'City CO2 Rounded Adjusted' : 'City CO2',
                            'Hwy CO2 Rounded Adjusted' : 'Highway CO2',
                            'Comb CO2 Rounded Adjusted (as shown on FE Label)' : 'Combined CO2'}, inplace = True)

In [9]:
df22

,Model Year,Mfr Name,Division,Carline,Verify Mfr Cd,Index (Model Type Index),Engine Displacement,# Cylinders,Transmission,City FE,...,City CO2,Highway CO2,Combined CO2,CO2-PHEV Composite Rounded Adjusted Combined CO2,240V Charge Time at 240 volts (hours),120V Charge time at 120 Volts (hours),PHEV Total Driving Range (rounded to nearest 10 miles)DISTANCE,City PHEV Composite MPGe,Hwy PHEV Composite MPGe,Comb PHEV Composite MPGe
0,2022,Honda,Acura,NSX,HNX,521,3.5,6,Auto(AM-S9),21,...,420,406,420,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,aston martin,Aston Martin Lagonda Ltd,Vantage V8,ASX,4,4.0,8,Auto(A8),18,...,494,364,435,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,Volkswagen Group of,Audi,R8 Coupe,VGA,14,5.2,10,Auto(AM-S7),13,...,670,453,572,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022,Volkswagen Group of,Audi,R8 Coupe RWD,VGA,16,5.2,10,Auto(AM-S7),14,...,627,385,518,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022,Volkswagen Group of,Audi,R8 Spyder,VGA,15,5.2,10,Auto(AM-S7),13,...,670,453,572,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,2022,Toyota,TOYOTA,HIGHLANDER HYBRID AWD,TYX,110,2.5,4,Auto(AV-S6),35,...,252,256,254,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1147,2022,Toyota,TOYOTA,HIGHLANDER HYBRID AWD LTD/PLAT,TYX,105,2.5,4,Auto(AV-S6),35,...,248,259,253,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1148,2022,Toyota,TOYOTA,SEQUOIA 4WD,TYX,97,5.7,8,Auto(S6),13,...,697,522,618,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1149,2022,Volvo,"Volvo Cars of North America, LLC",XC90 T5 AWD,VVX,229,2.0,4,Auto(S8),20,...,433,314,380,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#Drop the irrelevant columns from the later datasets. We will keep the same columns as in the 2015 dataset
for i in frames:
    for col in i.columns.tolist():
        if col not in columns:
            try:
                i.drop(col, axis = 1, inplace = True)
            except:
                print('Not successful at frame {0} and column {1}'.format(i, col))
            

In [11]:
df22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1151 entries, 0 to 1150
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Model Year                1151 non-null   int64         
 1   Mfr Name                  1151 non-null   object        
 2   Division                  1151 non-null   object        
 3   Carline                   1151 non-null   object        
 4   Engine Displacement       1151 non-null   float64       
 5   # Cylinders               1151 non-null   int64         
 6   Transmission              1151 non-null   object        
 7   City FE                   1151 non-null   int64         
 8   Highway FE                1151 non-null   int64         
 9   Combined FE               1151 non-null   int64         
 10  Air Aspiration Method     687 non-null    object        
 11  Transmission Description  1151 non-null   object        
 12  # Gears             

In [13]:
#Now we can aggregate the data, as we have dropped the unnecessarry columns from the dataframes and also use the same attribute names

df = pd.concat([df15, df16, df17, df18, df19, df20, df21, df22], ignore_index = True, axis = 0)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9248 entries, 0 to 9247
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Model Year                9248 non-null   int64         
 1   Mfr Name                  9248 non-null   object        
 2   Division                  9248 non-null   object        
 3   Carline                   9248 non-null   object        
 4   Engine Displacement       9248 non-null   float64       
 5   # Cylinders               9248 non-null   int64         
 6   Transmission              9248 non-null   object        
 7   City FE                   9248 non-null   int64         
 8   Highway FE                9248 non-null   int64         
 9   Combined FE               9248 non-null   int64         
 10  Air Aspiration Method     5517 non-null   object        
 11  Transmission Description  9248 non-null   object        
 12  # Gears             

In [14]:
#Now we want to rename the columns so it would match with the attribute names in the data warehouse
try:
    df.rename(columns = {"Model Year" : "Model_Year",
                        "Mfr Name" : "Mfr_Name",
                        "Engine Displacement" : "Engine_Displacement",
                        "# Cylinders" : "Cylinders",
                        "City FE" : "City_FE",
                        "Highway FE" : "Highway_FE",
                        "Combined FE" : "Combined_FE",
                        "Air Aspiration Method" : "Air_Aspiration_Method",
                        "Transmission Description" : "Trans_Desc",
                        "# Gears" : "Gears",
                        "Drive Desc" : "Drive_Desc",
                        "Carline Class Desc" : "Carline_Class_Desc",
                        "Release Date" : "Release_Date",
                        "City CO2" : "City_CO2",
                        "Highway CO2" : "Highway_CO2",
                        "Combined CO2" : "Combined_CO2"}, inplace = True)
except:
    print("Renaming failed. Check if column exists.")

#Let's verify if the renaming was successful
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9248 entries, 0 to 9247
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Model_Year             9248 non-null   int64         
 1   Mfr_Name               9248 non-null   object        
 2   Division               9248 non-null   object        
 3   Carline                9248 non-null   object        
 4   Engine_Displacement    9248 non-null   float64       
 5   Cylinders              9248 non-null   int64         
 6   Transmission           9248 non-null   object        
 7   City_FE                9248 non-null   int64         
 8   Highway_FE             9248 non-null   int64         
 9   Combined_FE            9248 non-null   int64         
 10  Air_Aspiration_Method  5517 non-null   object        
 11  Trans_Desc             9248 non-null   object        
 12  Gears                  9248 non-null   int64         
 13  Dri

## 2. Drop Nan Values

In [15]:
#The only column with NA values is the Air_Aspiration_Method. However, these NA values are just an alias for "Naturally Aspirated". Therefore, we will replace the NA values with "Naturally Aspirated"

df.fillna(value = 'Naturally Aspirated', inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9248 entries, 0 to 9247
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Model_Year             9248 non-null   int64         
 1   Mfr_Name               9248 non-null   object        
 2   Division               9248 non-null   object        
 3   Carline                9248 non-null   object        
 4   Engine_Displacement    9248 non-null   float64       
 5   Cylinders              9248 non-null   int64         
 6   Transmission           9248 non-null   object        
 7   City_FE                9248 non-null   int64         
 8   Highway_FE             9248 non-null   int64         
 9   Combined_FE            9248 non-null   int64         
 10  Air_Aspiration_Method  9248 non-null   object        
 11  Trans_Desc             9248 non-null   object        
 12  Gears                  9248 non-null   int64         
 13  Dri

In [16]:
df.tail()

,Model_Year,Mfr_Name,Division,Carline,Engine_Displacement,Cylinders,Transmission,City_FE,Highway_FE,Combined_FE,Air_Aspiration_Method,Trans_Desc,Gears,Drive_Desc,Carline_Class_Desc,Release_Date,City_CO2,Highway_CO2,Combined_CO2
9243,2022,Toyota,TOYOTA,HIGHLANDER HYBRID AWD,2.5,4,Auto(AV-S6),35,35,35,Naturally Aspirated,Selectable Continuously Variable (e.g. CVT wit...,6,All Wheel Drive,Standard SUV 4WD,2021-10-07,252,256,254
9244,2022,Toyota,TOYOTA,HIGHLANDER HYBRID AWD LTD/PLAT,2.5,4,Auto(AV-S6),35,34,35,Naturally Aspirated,Selectable Continuously Variable (e.g. CVT wit...,6,All Wheel Drive,Standard SUV 4WD,2021-10-07,248,259,253
9245,2022,Toyota,TOYOTA,SEQUOIA 4WD,5.7,8,Auto(S6),13,17,14,Naturally Aspirated,Semi-Automatic,6,Part-time 4-Wheel Drive,Standard SUV 4WD,2021-08-04,697,522,618
9246,2022,Volvo,"Volvo Cars of North America, LLC",XC90 T5 AWD,2.0,4,Auto(S8),20,28,23,TC,Semi-Automatic,8,All Wheel Drive,Standard SUV 4WD,2021-07-02,433,314,380
9247,2022,Volvo,"Volvo Cars of North America, LLC",XC90 T6 AWD,2.0,4,Auto(S8),19,27,22,TS,Semi-Automatic,8,All Wheel Drive,Standard SUV 4WD,2021-07-02,464,333,405


## 3. Clean the categorical values

## a) Transmission column

In [17]:
print(df['Transmission'].unique())

['Auto' 'Manual' 'Auto(AM6)' 'Auto(AM7)' 'Manual(M6)' 'Auto(AM-S6)'
 'Auto(AM-S7)' 'Auto(S8)' 'Manual(M7)' 'Auto(AV-S7)' 'Auto(S6)' 'Auto(A7)'
 'Auto(A9)' 'Manual(M5)' 'Auto(S7)' 'Auto(A6)' 'Auto(AV)' 'Auto(AM8)'
 'Auto(AM-S8)' 'Auto(S9)' 'Auto(AV-S8)' 'Auto(AV-S6)' 'Auto(A4)'
 'Auto(A8)' 'Auto(A5)' 'Auto(S5)' 'Auto(AM-S9)' 'Auto(S10)' 'Auto(A10)'
 'Auto(AV-S10)' 'Auto(A1)' 'Auto(AV-S1)']


In [18]:
#Transform all the Transmission methods to only 2 types: auto and manual

auto = []
manual = []
for i in df['Transmission'].unique().tolist():
    if i.startswith("Auto"):
        auto.append(i)
    elif i.startswith("Manual"):
        manual.append(i)

df["Transmission"].replace(to_replace = auto, value = "Auto", inplace = True)
df["Transmission"].replace(to_replace = manual, value = "Manual", inplace = True)

print(df['Transmission'].unique())

['Auto' 'Manual']


In [19]:
df.tail()

,Model_Year,Mfr_Name,Division,Carline,Engine_Displacement,Cylinders,Transmission,City_FE,Highway_FE,Combined_FE,Air_Aspiration_Method,Trans_Desc,Gears,Drive_Desc,Carline_Class_Desc,Release_Date,City_CO2,Highway_CO2,Combined_CO2
9243,2022,Toyota,TOYOTA,HIGHLANDER HYBRID AWD,2.5,4,Auto,35,35,35,Naturally Aspirated,Selectable Continuously Variable (e.g. CVT wit...,6,All Wheel Drive,Standard SUV 4WD,2021-10-07,252,256,254
9244,2022,Toyota,TOYOTA,HIGHLANDER HYBRID AWD LTD/PLAT,2.5,4,Auto,35,34,35,Naturally Aspirated,Selectable Continuously Variable (e.g. CVT wit...,6,All Wheel Drive,Standard SUV 4WD,2021-10-07,248,259,253
9245,2022,Toyota,TOYOTA,SEQUOIA 4WD,5.7,8,Auto,13,17,14,Naturally Aspirated,Semi-Automatic,6,Part-time 4-Wheel Drive,Standard SUV 4WD,2021-08-04,697,522,618
9246,2022,Volvo,"Volvo Cars of North America, LLC",XC90 T5 AWD,2.0,4,Auto,20,28,23,TC,Semi-Automatic,8,All Wheel Drive,Standard SUV 4WD,2021-07-02,433,314,380
9247,2022,Volvo,"Volvo Cars of North America, LLC",XC90 T6 AWD,2.0,4,Auto,19,27,22,TS,Semi-Automatic,8,All Wheel Drive,Standard SUV 4WD,2021-07-02,464,333,405


## b) Air Aspiration Column

In [20]:
#TC, SC, TS and OT signify "Turbocharged", "Supercharged", "Turbocharged+Supercharged" and "Other"

print(df["Air_Aspiration_Method"].unique())

['Naturally Aspirated' 'Turbocharged' 'Supercharged'
 'Turbocharged+Supercharged' 'TC' 'SC' 'TS' 'OT']


In [21]:
#We want to replace the TC, SC, TS and OT with their full description for readability

df["Air_Aspiration_Method"].replace(to_replace = ("TC", "SC", "TS", "OT"), value = ('Turbocharged', 'Supercharged', 'Turbocharged+Supercharged', 'Other'), inplace = True)

print(df["Air_Aspiration_Method"].unique())

['Naturally Aspirated' 'Turbocharged' 'Supercharged'
 'Turbocharged+Supercharged' 'Other']


## c) Trans Desc column

In [22]:
# Let's check now the transmission description

print(df["Trans_Desc"].unique())


['Automated Manual' 'Manual'
 'Automated Manual- Selectable (e.g. Automated Manual with paddles)'
 'Semi-Automatic' 'Automatic' 'Continuously Variable'
 'Selectable Continuously Variable (e.g. CVT with paddles)']


In [23]:
# We want to remove the examples from the values in the transmission description column

df["Trans_Desc"].replace(to_replace = ("Automated Manual- Selectable (e.g. Automated Manual with paddles)", "Selectable Continuously Variable (e.g. CVT with paddles)"),
                        value = ("Automated Manual-Selectable", "Selectable Continuously Variable"), inplace = True)

print(df["Trans_Desc"].unique())

['Automated Manual' 'Manual' 'Automated Manual-Selectable'
 'Semi-Automatic' 'Automatic' 'Continuously Variable'
 'Selectable Continuously Variable']


## d) Drive Desc column

In [24]:
#Looking at the drive description column we can notice a specific values "d"

print(df["Drive_Desc"].unique())

['2-Wheel Drive, Rear' 'All Wheel Drive' '2-Wheel Drive, Front' 'd'
 '4-Wheel Drive' 'Part-time 4-Wheel Drive']


In [25]:
#There is one row containing this value "d", which can be safely dropped

df[df["Drive_Desc"] == "d"]

,Model_Year,Mfr_Name,Division,Carline,Engine_Displacement,Cylinders,Transmission,City_FE,Highway_FE,Combined_FE,Air_Aspiration_Method,Trans_Desc,Gears,Drive_Desc,Carline_Class_Desc,Release_Date,City_CO2,Highway_CO2,Combined_CO2
62,2015,Porsche,Porsche,Boxster GTS,3.4,6,Manual,19,26,22,Naturally Aspirated,Manual,6,d,Two Seaters,2014-06-23,458,334,402


In [26]:
#Drop the specific row which contains value "d" in the "Drive_Desc" column

df.drop(df[df["Drive_Desc"] == "d"].index, axis = 0, inplace = True)

In [27]:
#Check if the drop was successful

print(df["Drive_Desc"].unique())

['2-Wheel Drive, Rear' 'All Wheel Drive' '2-Wheel Drive, Front'
 '4-Wheel Drive' 'Part-time 4-Wheel Drive']


# 4. Load the data

In [28]:
#Establish the connection between the datawarehouse and the python file

olap = create_engine('mysql+pymysql://root:''@localhost/marketing')
conn = olap.connect()

## a) Engine data

In [29]:
#Create an engine dataframe that would be later loaded into the engine table in the data warehouse

df_engine = df[["Engine_Displacement", "Cylinders", "Air_Aspiration_Method"]]
df_engine.head()


,Engine_Displacement,Cylinders,Air_Aspiration_Method
0,6.0,12,Naturally Aspirated
1,4.7,8,Naturally Aspirated
2,4.7,8,Naturally Aspirated
3,4.7,8,Naturally Aspirated
4,4.7,8,Naturally Aspirated


In [30]:
#Select the unique combinations of columns in the engine dataframe

df_engine = df_engine.groupby(["Engine_Displacement", "Cylinders", "Air_Aspiration_Method"]).size().reset_index().rename(columns = {0 : "Count"})
df_engine

,Engine_Displacement,Cylinders,Air_Aspiration_Method,Count
0,0.9,3,Turbocharged,6
1,1.0,3,Naturally Aspirated,2
2,1.0,3,Turbocharged,12
3,1.0,4,Turbocharged,1
4,1.2,3,Naturally Aspirated,26
...,...,...,...,...
93,6.7,12,Turbocharged,22
94,6.8,8,Turbocharged,7
95,7.0,8,Naturally Aspirated,1
96,8.0,16,Turbocharged,10


In [31]:
#We will drop the Count column as we do not need it in the Engine table

df_engine.drop("Count", axis = 1, inplace = True)
df_engine

,Engine_Displacement,Cylinders,Air_Aspiration_Method
0,0.9,3,Turbocharged
1,1.0,3,Naturally Aspirated
2,1.0,3,Turbocharged
3,1.0,4,Turbocharged
4,1.2,3,Naturally Aspirated
...,...,...,...
93,6.7,12,Turbocharged
94,6.8,8,Turbocharged
95,7.0,8,Naturally Aspirated
96,8.0,16,Turbocharged


In [32]:
# Lastly, we will change the index numbers so they would start from 1 to 130 and match the indexes from the engine table in the data warehouse

df_engine.index += 1
df_engine

,Engine_Displacement,Cylinders,Air_Aspiration_Method
1,0.9,3,Turbocharged
2,1.0,3,Naturally Aspirated
3,1.0,3,Turbocharged
4,1.0,4,Turbocharged
5,1.2,3,Naturally Aspirated
...,...,...,...
94,6.7,12,Turbocharged
95,6.8,8,Turbocharged
96,7.0,8,Naturally Aspirated
97,8.0,16,Turbocharged


In [33]:
#Load it to the engine table in the data warehouse

df_engine.to_sql("engine", conn, if_exists = "append", index = False)

98

## b) Transmission Data

In [34]:
#Get the data for the Transmission table

df_trans = df[['Transmission', 'Trans_Desc', 'Gears']]
df_trans

,Transmission,Trans_Desc,Gears
0,Auto,Automated Manual,7
1,Auto,Automated Manual,7
2,Manual,Manual,6
3,Auto,Automated Manual,7
4,Manual,Manual,6
...,...,...,...
9243,Auto,Selectable Continuously Variable,6
9244,Auto,Selectable Continuously Variable,6
9245,Auto,Semi-Automatic,6
9246,Auto,Semi-Automatic,8


In [35]:
#Get unique combinations of the columns

df_trans = df_trans.groupby(['Transmission', 'Trans_Desc', 'Gears']).size().reset_index().rename(columns = {0 : "Counts"})
df_trans

,Transmission,Trans_Desc,Gears,Counts
0,Auto,Automated Manual,5,2
1,Auto,Automated Manual,6,65
2,Auto,Automated Manual,7,220
3,Auto,Automated Manual,8,24
4,Auto,Automated Manual-Selectable,6,100
5,Auto,Automated Manual-Selectable,7,523
6,Auto,Automated Manual-Selectable,8,157
7,Auto,Automated Manual-Selectable,9,6
8,Auto,Automatic,1,2
9,Auto,Automatic,4,18


In [36]:
# Drop the Counts column and increment indexes is 1

df_trans.drop('Counts', axis = 1, inplace = True)
df_trans.index += 1

df_trans

,Transmission,Trans_Desc,Gears
1,Auto,Automated Manual,5
2,Auto,Automated Manual,6
3,Auto,Automated Manual,7
4,Auto,Automated Manual,8
5,Auto,Automated Manual-Selectable,6
6,Auto,Automated Manual-Selectable,7
7,Auto,Automated Manual-Selectable,8
8,Auto,Automated Manual-Selectable,9
9,Auto,Automatic,1
10,Auto,Automatic,4


In [37]:
# Load the data into the Transmission table in the data warehouse

df_trans.to_sql('transmission', conn, if_exists = 'append', index = False)

31

## c) Car Data

In [38]:
#Select the data for the Car table in the data warehouse

df_car = df[["Model_Year", "Mfr_Name", "Division", "Carline", "Drive_Desc", "Carline_Class_Desc", "Release_Date"]]
df_car

,Model_Year,Mfr_Name,Division,Carline,Drive_Desc,Carline_Class_Desc,Release_Date
0,2015,aston martin,Aston Martin Lagonda Ltd,V12 Vantage S,"2-Wheel Drive, Rear",Two Seaters,2014-01-20
1,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage,"2-Wheel Drive, Rear",Two Seaters,2014-05-19
2,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage,"2-Wheel Drive, Rear",Two Seaters,2014-05-19
3,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage S,"2-Wheel Drive, Rear",Two Seaters,2014-05-19
4,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage S,"2-Wheel Drive, Rear",Two Seaters,2014-05-19
...,...,...,...,...,...,...,...
9243,2022,Toyota,TOYOTA,HIGHLANDER HYBRID AWD,All Wheel Drive,Standard SUV 4WD,2021-10-07
9244,2022,Toyota,TOYOTA,HIGHLANDER HYBRID AWD LTD/PLAT,All Wheel Drive,Standard SUV 4WD,2021-10-07
9245,2022,Toyota,TOYOTA,SEQUOIA 4WD,Part-time 4-Wheel Drive,Standard SUV 4WD,2021-08-04
9246,2022,Volvo,"Volvo Cars of North America, LLC",XC90 T5 AWD,All Wheel Drive,Standard SUV 4WD,2021-07-02


In [39]:
#Find all the possible combinations of columns for the car table

df_car = df_car.groupby(["Model_Year", "Mfr_Name", "Division", "Carline", "Drive_Desc", "Carline_Class_Desc", "Release_Date"]).size().reset_index().rename(columns = {0 : "Counts"})
df_car

,Model_Year,Mfr_Name,Division,Carline,Drive_Desc,Carline_Class_Desc,Release_Date,Counts
0,2015,BMW,BMW,228i,"2-Wheel Drive, Rear",Subcompact Cars,2014-07-15,1
1,2015,BMW,BMW,228i,"2-Wheel Drive, Rear",Subcompact Cars,2014-07-16,1
2,2015,BMW,BMW,228i xDrive,All Wheel Drive,Subcompact Cars,2014-07-16,1
3,2015,BMW,BMW,320i,"2-Wheel Drive, Rear",Compact Cars,2014-07-14,1
4,2015,BMW,BMW,320i xDrive,All Wheel Drive,Compact Cars,2014-07-16,1
...,...,...,...,...,...,...,...,...
7064,2022,aston martin,Aston Martin Lagonda Ltd,DB11 V12,"2-Wheel Drive, Rear",Minicompact Cars,2021-11-29,1
7065,2022,aston martin,Aston Martin Lagonda Ltd,DB11 V8,"2-Wheel Drive, Rear",Minicompact Cars,2021-10-25,1
7066,2022,aston martin,Aston Martin Lagonda Ltd,DBS,"2-Wheel Drive, Rear",Minicompact Cars,2021-11-29,1
7067,2022,aston martin,Aston Martin Lagonda Ltd,DBX V8,4-Wheel Drive,Standard SUV 4WD,2021-10-25,1


In [40]:
#Increment the index by 1 so it would match the index from the car table in the data warehouse and also drop the Counts column

df_car.index += 1
df_car.drop('Counts', axis = 1, inplace = True)
df_car

,Model_Year,Mfr_Name,Division,Carline,Drive_Desc,Carline_Class_Desc,Release_Date
1,2015,BMW,BMW,228i,"2-Wheel Drive, Rear",Subcompact Cars,2014-07-15
2,2015,BMW,BMW,228i,"2-Wheel Drive, Rear",Subcompact Cars,2014-07-16
3,2015,BMW,BMW,228i xDrive,All Wheel Drive,Subcompact Cars,2014-07-16
4,2015,BMW,BMW,320i,"2-Wheel Drive, Rear",Compact Cars,2014-07-14
5,2015,BMW,BMW,320i xDrive,All Wheel Drive,Compact Cars,2014-07-16
...,...,...,...,...,...,...,...
7065,2022,aston martin,Aston Martin Lagonda Ltd,DB11 V12,"2-Wheel Drive, Rear",Minicompact Cars,2021-11-29
7066,2022,aston martin,Aston Martin Lagonda Ltd,DB11 V8,"2-Wheel Drive, Rear",Minicompact Cars,2021-10-25
7067,2022,aston martin,Aston Martin Lagonda Ltd,DBS,"2-Wheel Drive, Rear",Minicompact Cars,2021-11-29
7068,2022,aston martin,Aston Martin Lagonda Ltd,DBX V8,4-Wheel Drive,Standard SUV 4WD,2021-10-25


In [41]:
df_car.to_sql('car', conn, if_exists = 'append', index = False)

7069

## d) FE_Facts data

In [44]:
#First we will drop from the dataframe the columns which cannot be found in the fact table

df_fact = df.copy()
df_fact.drop(["Model_Year", "Mfr_Name", "Division", "Carline", "Drive_Desc", "Carline_Class_Desc", "Release_Date",
               'Transmission', 'Trans_Desc', 'Gears', "Engine_Displacement", "Cylinders", "Air_Aspiration_Method"], axis = 1, inplace = True)
df_fact

,City_FE,Highway_FE,Combined_FE,City_CO2,Highway_CO2,Combined_CO2
0,12,18,14,742,488,628
1,14,21,16,655,426,552
2,13,19,15,681,461,582
3,14,21,16,655,426,552
4,13,19,15,681,461,582
...,...,...,...,...,...,...
9243,35,35,35,252,256,254
9244,35,34,35,248,259,253
9245,13,17,14,697,522,618
9246,20,28,23,433,314,380


In [80]:
# Now for each row in the initial dataframe, we will find the respective indices in the dimensional tables and add them in lists 

car_idx = []
engine_idx = []
trans_idx = []
for idx, row in df.iterrows():

    temp = df_car[df_car["Model_Year"] == row["Model_Year"]]
    temp = temp[temp["Mfr_Name"] == row["Mfr_Name"]]
    temp = temp[temp["Division"] == row["Division"]]
    temp = temp[temp["Carline"] == row["Carline"]]
    temp = temp[temp["Drive_Desc"] == row["Drive_Desc"]]
    temp = temp[temp["Carline_Class_Desc"] == row["Carline_Class_Desc"]]
    temp = temp[temp["Release_Date"] == row["Release_Date"]]
    car_idx.extend(temp.index.values)


    temp = df_engine[df_engine["Engine_Displacement"] == row["Engine_Displacement"]]
    temp = temp[temp["Cylinders"] == row["Cylinders"]]
    temp = temp[temp["Air_Aspiration_Method"] == row["Air_Aspiration_Method"]]
    engine_idx.extend(temp.index.values)

    temp = df_trans[df_trans["Transmission"] == row["Transmission"]]
    temp = temp[temp["Trans_Desc"] == row["Trans_Desc"]]
    temp = temp[temp["Gears"] == row["Gears"]]
    trans_idx.extend(temp.index.values)



In [81]:
# Append the indeces to the fact table

df_fact["Model_ID"] = car_idx
df_fact["Engine_ID"] = engine_idx
df_fact["Trans_ID"] = trans_idx

df_fact

,City_FE,Highway_FE,Combined_FE,City_CO2,Highway_CO2,Combined_CO2,Model_ID,Engine_ID,Trans_ID
0,12,18,14,742,488,628,505,85,3
1,14,21,16,655,426,552,506,67,3
2,13,19,15,681,461,582,506,67,30
3,14,21,16,655,426,552,507,67,3
4,13,19,15,681,461,582,507,67,30
...,...,...,...,...,...,...,...,...,...
9243,35,35,35,252,256,254,6943,27,19
9244,35,34,35,248,259,253,6944,27,19
9245,13,17,14,697,522,618,6954,82,24
9246,20,28,23,433,314,380,7063,20,26


In [82]:
# Load the data to the fact table in the data warehouse

df_fact.to_sql('fe_facts', conn, if_exists = 'append', index = False)

9247